In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import numpy as np
import random
import cv2
import os
import glob

In [2]:
data = []
labels = []

In [3]:
image_files = [f for f in glob.glob(r'C:\Users\ii\Jupyter-Notebook\gender-detection\gender_dataset_face'+'\**\*', recursive=True) if not os.path.isdir(f)]
random.shuffle(image_files)

#print(image_files)

In [4]:
for img in image_files:
    image = cv2.imread(img)
    image = cv2.resize(image, (96,96))
    image = img_to_array(image)
    data.append(image)
    label = img.split(os.path.sep)[-2]
    if label == "woman":
        label = 1
    else:
        label = 0
        
    labels.append([label])


In [5]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [6]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2,
                                                  random_state=42)

trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")


In [7]:
def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    if K.image_data_format() == "channels_first":
        inputShape = (depth, height, width)
        chanDim = 1

    model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(classes))
    model.add(Activation("sigmoid"))

    return model

In [ ]:
epochs = 100
batch_size = 64
img_dims = (96,96,3)

model = build(width=img_dims[0], height=img_dims[1], depth=img_dims[2],
                            classes=2)

opt = Adam(lr=1e-3, decay=1e-3/epochs)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
                        validation_data=(testX,testY),
                        steps_per_epoch=len(trainX) // batch_size,
                        epochs=epochs, verbose=1)

C:\Users\ii\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\ii\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
28/28 [==============================] - 48s 1s/step - loss: 0.7508 - accuracy: 0.7131 - val_loss: 0.6526 - val_accuracy: 0.6234
Epoch 2/100
28/28 [==============================] - 33s 1s/step - loss: 0.5435 - accuracy: 0.7996 - val_loss: 0.7025 - val_accuracy: 0.5541
Epoch 3/100
28/28 [==============================] - 33s 1s/step - loss: 0.4253 - accuracy: 0.8226 - val_loss: 1.7278 - val_accuracy: 0.5195
Epoch 4/100
28/28 [==============================] - 34s 1s/step - loss: 0.3840 - accuracy: 0.8596 - val_loss: 1.7303 - val_accuracy: 0.5195
Epoch 5/100
28/28 [==============================] - 35s 1s/step - loss: 0.3305 - accuracy: 0.8770 - val_loss: 1.7412 - val_accuracy: 0.5195
Epoch 6/100
28/28 [==============================] - 33s 1s/step - loss: 0.3503 - accuracy: 0.8675 - val_loss: 1.6449 - val_accuracy: 0.5195
Epoch 7/100
28/28 [==============================] - 35s 1s/step - loss: 0.3074 - accuracy: 0.8849 - val_loss: 1.2042 - val_accuracy: 0.5216
Epoch 8/100
2

In [ ]:
model.save('gender_detection.model')